In [238]:
import pandas as pd
import numpy as np

import seaborn as sns
sns.set(style="ticks", color_codes=True)
current_palette = sns.color_palette()
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
from scipy.stats.mstats import mode
from scipy.stats import chisquare
from scipy.spatial.distance import jaccard


from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer, StandardScaler

import sklearn.preprocessing, sklearn.decomposition,\
       sklearn.linear_model, sklearn.pipeline, sklearn.metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn_pandas import gen_features

import time
import datetime
from datetime import date, datetime, timedelta

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)

import os
import pickle


### 1 Import and quick EDA

In [173]:
cwd = os.getcwd()
cwd

'C:\\Users\\mstre\\Dropbox\\0Big_data\\Xpdia\\Mathieu'

In [ ]:
###### import df

In [239]:
ds2 = pd.read_csv('ds1.csv', sep = "|")
ds2.head(3)

,date_time,site_name,user_location_country,user_location_region,user_location_city,user_location_latitude,user_location_longitude,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,hotel_country,is_booking,hotel_id,prop_is_branded,prop_starrating,distance_band,hist_price_band,popularity_band,cnt,date_ss,date_ci,date_co,month_ss,day_ss,hour_ss,month_ci,day_ci,month_co,day_co,date_ss1,month_ci1,month_co1,season_cat,LOS,LOS_range,LOS_cat,is_satsun_ss,is_worktime_ss,is_satsun,is_monfri,BW_ss,BW_ss_range,BW_ss_cat,city_state,region_dest,user_cat,channel_cat,state,2015_hh_med_inc_state,income_hh,income_hh_range,income_hh_cat,USD_id,USD_ids,is_churn,is_session,is_prebooking,session_cat,is_churns,is_prebookings,session_cats,year_rank_ss,season_dest_rank_ss,timedelta_ss,package_income_t,mobile_BW_t,hotel_band,cluster_id
0,2015-01-01 00:03:00,EXPEDIA.COM,UNITED STATES OF AMERICA,OH,WEST CHESTER,39.328640,-84.425730,775.8321,-803007886,0,0,324,2015-01-22,2015-01-23,2,0,1,5527237,UNITED STATES OF AMERICA,0,153390727,1,3,C,H,H,1,2015-01-01 00:03:00,2015-01-22 00:00:00,2015-01-23 00:00:00,1,3,0,1.0,3.0,1.0,4.0,January,January,January,is_winter,1.0,"(0, 1]",0to1day,0,0,0,1,21.0,"(7, 30]",7to30days,"WEST CHESTER, OH",USA,familynokids_usa,mail,Ohio,51610,51610.0,"(27000, 56000]",27K_56K,-803007886 5527237,-803007886 5527237 is_winter,1.0,1,0.0,churn,1.0,0.0,1,1,1,365.0,0.0,0.0,H H C,72
1,2015-01-01 00:03:00,EXPEDIA.COM,UNITED STATES OF AMERICA,CA,SAN JOSE,37.334754,-121.905447,82.2199,-635508862,1,0,510,2015-01-01,2015-01-02,2,0,1,187717089,UNITED STATES OF AMERICA,0,54164532,1,3,F,M,H,1,2015-01-01 00:03:00,2015-01-01 00:00:00,2015-01-02 00:00:00,1,3,0,1.0,3.0,1.0,4.0,January,January,January,is_winter,1.0,"(0, 1]",0to1day,0,0,0,1,-0.0,NaN,0to1day,"SAN JOSE, CA",USA,familynokids_usa,google_srch,California,65087,34977.0,"(27000, 56000]",27K_56K,-635508862 187717089,-635508862 187717089 is_winter,1.0,1,0.0,churn,1.0,0.0,1,1,1,365.0,0.0,-0.0,M H F,54
2,2015-01-01 00:05:00,EXPEDIA.COM,UNITED STATES OF AMERICA,VA,RICHMOND,37.555881,-77.498330,2434.4907,1297850351,0,0,510,2015-01-05,2015-01-10,1,0,1,5527578,UNITED STATES OF AMERICA,0,27025644,0,2,C,VL,M,1,2015-01-01 00:05:00,2015-01-05 00:00:00,2015-01-10 00:00:00,1,3,0,1.0,0.0,1.0,5.0,January,January,January,is_winter,5.0,"(4, 375]",over4days,0,0,1,1,4.0,"(1, 7]",1to7days,"RICHMOND, VA",USA,single_usa,google_srch,Virginia,66916,26540.0,"(0, 27000]",less27K,1297850351 5527578,1297850351 5527578 is_winter,0.0,1,1.0,prebooking,0.0,1.0,2,1,1,365.0,0.0,0.0,VL M C,7


In [178]:
ds2.columns[ds2.isna().any()].tolist()
#ds2[ds2['BW_ss_range'] .isnull()] 

['user_location_latitude',
 'user_location_longitude',
 'orig_destination_distance',
 'LOS_range',
 'BW_ss_range']

In [ ]:
###### check the distribution of the target class y 

# we have unbalances classes between booking, churns and prebookings
# aka H1: "is booking" only 8.56% on average.

#Why is this a problem? Model biased towards predicting no booking would return:
# H0: 91.44% accuracy by simply predicting “no booking” (churns or prebookings)

#ds2['session_cats'].plot(kind='bar')  # .value_counts()
#ds2['session_cats'].describe()

In [ ]:
# address class imbalance

# OPTION 1: Rebalance classes  in the training dataset (H1 is booking only 8.56% on average)  
Split the training dataset into K-folds. 
Split training between training and validation 
Apply using SMOTE, Synthetic Minority Oversampling Technique. 

# OPTION 2: Use stratified K-fold (preferred)

In [ ]:
###### # check FEATURES 
# Check pearson s coefficient – pairplot correlation. If highly correlated, perform Anova.

sns.pairplot(ds2)
plt.figure(figsize=(20,18))
sns.heatmap(ds_ML1.corr(), annot=True)

In [177]:
# issue, I need numeric for y 
# Find all correlations and sort 
#correlations_data = ds2.corr()['session_cats'].sort_values()

# Print the most negative correlations
#print(correlations_data.head(15), '\n')

# Print the most positive correlations
#print(correlations_data.tail(15))

### 2 Preparation for ML analysis

In [196]:
## Remove features
# irrelevant
# features with missing data 
# duplicate features
remove_col = []

remove_col = ['date_time','site_name','user_location_country','user_location_region','user_location_city',\
              'user_location_latitude', 'user_location_longitude', 'orig_destination_distance','channel',\
              'srch_ci','srch_co','hotel_country','date_ss','date_ci','date_ss1','date_ci','date_co','date_ss1',\
              'month_ci1','month_co1','LOS_range','BW_ss_range','city_state','state','2015_hh_med_inc_state',\
              'income_hh_range','USD_id','USD_ids','is_churn','is_session','is_prebooking','session_cat','hotel_band',\
           'cluster_id','is_churns','is_prebookings', 'is_booking'   
          ]
ds2.drop(remove_col, inplace=True, axis=1)


In [197]:
ds2.head(1)

,user_id,is_mobile,is_package,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,is_booking,hotel_id,prop_is_branded,prop_starrating,distance_band,hist_price_band,popularity_band,cnt,month_ss,day_ss,hour_ss,month_ci,day_ci,month_co,day_co,season_cat,LOS,LOS_cat,is_satsun_ss,is_worktime_ss,is_satsun,is_monfri,BW_ss,BW_ss_cat,region_dest,user_cat,channel_cat,income_hh,income_hh_cat,is_churns,is_prebookings,year_rank_ss,season_dest_rank_ss,timedelta_ss,package_income_t,mobile_BW_t
0,-803007886,0,0,2,0,1,5527237,0,153390727,1,3,C,H,H,1,1,3,0,1.0,3.0,1.0,4.0,is_winter,1.0,0to1day,0,0,0,1,21.0,7to30days,USA,familynokids_usa,mail,51610.0,27K_56K,1.0,0.0,1,1,365.0,0.0,0.0


In [198]:
# reorder columns to put 'is_booking', 'is_churns', 'is_prebookings' at the end (y)
list(ds2)
cols=['user_id',
 'is_mobile','is_package','srch_adults_cnt','srch_children_cnt','srch_rm_cnt','srch_destination_id',
 'hotel_id','prop_is_branded','prop_starrating','distance_band','hist_price_band','popularity_band',
 'cnt','month_ss','day_ss','hour_ss','month_ci','day_ci','month_co','day_co','season_cat','LOS','LOS_cat',
 'is_satsun_ss','is_worktime_ss','is_satsun','is_monfri','BW_ss','BW_ss_cat','region_dest','user_cat',
 'channel_cat','income_hh','income_hh_cat','year_rank_ss','season_dest_rank_ss','timedelta_ss',
 'package_income_t','mobile_BW_t','session_cats'  
     ]
ds2 = ds2[cols]

In [199]:
#remove categories which are duplicates of numeric (option)
remove_col = []
remove_col = ['income_hh_cat','LOS_cat','BW_ss_cat']
ds2.drop(remove_col, inplace=True, axis=1)

In [200]:
ds2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 936378 entries, 0 to 936377
Data columns (total 40 columns):
user_id                936378 non-null int64
is_mobile              936378 non-null int64
is_package             936378 non-null int64
srch_adults_cnt        936378 non-null int64
srch_children_cnt      936378 non-null int64
srch_rm_cnt            936378 non-null int64
srch_destination_id    936378 non-null int64
hotel_id               936378 non-null int64
prop_is_branded        936378 non-null int64
prop_starrating        936378 non-null int64
distance_band          936378 non-null object
hist_price_band        936378 non-null object
popularity_band        936378 non-null object
cnt                    936378 non-null int64
month_ss               936378 non-null int64
day_ss                 936378 non-null int64
hour_ss                936378 non-null int64
month_ci               936378 non-null float64
day_ci                 936378 non-null float64
month_co               9363

In [183]:
## Transform all features into numeric or binary for ML

# 1 one hot encoder: 
# Encode categorical integer features using a one-hot aka one-of-K scheme.
# the input to this transformer should be a matrix of integers
# denoting the values taken on by categorical (discrete) features from 1 to n.

# label_binarizer:
# a one-hot encoding of y labels should use a LabelBinarizer instead
# needed to convert multi-class labels to binary labels (belong or does not belong to the class).
# LabelBinarizer makes this process easy with the transform method.

# label_binarize for fixed set of class labels known ahead of time.
          
features = [('user_id',                   None),       # remove?
            ('is_mobile',                 None),
            ('is_package',                None),
            ('srch_adults_cnt',         None),          # [StandardScaler()])
            ('srch_children_cnt',       None),          # [StandardScaler()])
            ('srch_rm_cnt',             None),          # [StandardScaler()])
           # ('srch_destination_id',       None),        # remove?
           # ('hotel_id',                  None),        # remove?
            ('prop_is_branded',           None),
            ('prop_starrating',[LabelBinarizer()]),
            ('distance_band',  [LabelBinarizer()]),
            ('hist_price_band',[LabelBinarizer()]),
            ('popularity_band',[LabelBinarizer()]),
            ('cnt',                     None),          # [StandardScaler()])
            ('month_ss',                None),          # [StandardScaler()])
            ('day_ss',                  None),          # [StandardScaler()])
            ('hour_ss',                 None),          # [StandardScaler()]), 
            ('month_ci',                None),          # [StandardScaler()])
            ('day_ci',                  None),          # [StandardScaler()])
            ('month_co',                None),          # [StandardScaler()])
            ('day_co',                  None),          # [StandardScaler()])
            ('season_cat',     [LabelBinarizer()]),
            ('LOS',                     None),          # [StandardScaler()])
            ('is_satsun_ss',              None),
            ('is_worktime_ss',            None),
            ('is_satsun',                 None),
            ('is_monfri',                 None),
            (['BW_ss'],     [StandardScaler()]),
            ('region_dest', [LabelBinarizer()]),
            ('user_cat',    [LabelBinarizer()]),
            ('channel_cat', [LabelBinarizer()]), 
            ('income_hh',              None),         #[StandardScaler()]),
            ('year_rank_ss',           None),         #[StandardScaler()]),
            ('season_dest_rank_ss',    None),         #[StandardScaler()]),
            ('timedelta_ss',           None),         #[StandardScaler()]),
            ('package_income_t',          None),
            ('mobile_BW_t',               None), 
            ('is_booking',                None),
            ('is_churns',                 None),
            ('is_prebookings',            None), 
            ]


mapper= DataFrameMapper(features, default=False,df_out=False) # mapper for ML analysis
mapperdf=DataFrameMapper(features, default=False,df_out=True) # mapper to split train and test
#ds_ML1=mapper1.fit_transform(X_train)
# note: use mapper1 and mapperdf1 instead to apply standardscaler 

In [168]:
mapper

DataFrameMapper(default=False, df_out=False,
        features=[('user_id', None), ('is_mobile', None), ('is_package', None), ('srch_adults_cnt', None), ('srch_children_cnt', None), ('srch_rm_cnt', None), ('srch_destination_id', None), ('hotel_id', None), ('prop_is_branded', None), ('prop_starrating', [LabelBinarizer(neg_label=0, pos_label=1, sparse_ou..., None), ('mobile_BW_t', None), ('is_booking', None), ('is_churns', None), ('is_prebookings', None)],
        input_df=False, sparse=False)

In [213]:
##### select season (MUST because of times series)
ds2_spring=ds2[ds2['season_cat']=='is_spring']
ds2_summer=ds2[ds2['season_cat']=='is_summer']
ds2_autumn=ds2[ds2['season_cat']=='is_autumn']
ds2_winter=ds2[ds2['season_cat']=='is_winter']

# create and save data to disk

ML1_season2=np.round(mapperdf.fit_transform(ds2_summer.copy()), 2) #need to choose one model
filename2 = 'ML1_season2.sav'
pickle.dump(ML1_season2, open(filename2, 'wb'))
print(ML1_season2,' saved')
    
'''
ML1_season1=np.round(mapperdf.fit_transform(ds2_spring.copy()), 2)
filename1 = 'ML1_season1.sav'
pickle.dump(ML1_season1, open(filename1, 'wb'))
print(ML1_season1,' saved')
    

ML1_season3=np.round(mapperdf.fit_transform(ds2_autumn.copy()), 2)
filename3 = 'ML1_season3.sav'
pickle.dump(ML1_season3, open(filename3, 'wb'))
print(ML1_season3,' saved')
        

ML1_season4=np.round(mapperdf.fit_transform(ds2_winter.copy()), 2)
filename4 = 'ML1_season4.sav'
pickle.dump(ML1_season4, open(filename4, 'wb'))
print(ML1_season4,' saved')
'''

             user_id  is_mobile  is_package  srch_adults_cnt  \
65     -1.825978e+09        0.0         1.0              2.0   
71     -1.490758e+09        0.0         0.0              5.0   
73     -1.825978e+09        0.0         1.0              2.0   
78     -1.490758e+09        0.0         0.0              3.0   
84     -1.825978e+09        0.0         1.0              2.0   
85     -1.490758e+09        0.0         0.0              3.0   
86     -1.490758e+09        0.0         0.0              3.0   
89     -1.962591e+09        1.0         0.0              2.0   
90      1.488102e+08        0.0         1.0              2.0   
91     -1.490758e+09        0.0         1.0              5.0   
92     -1.490758e+09        0.0         0.0              4.0   
93     -1.490758e+09        0.0         1.0              5.0   
94     -1.490758e+09        0.0         0.0              2.0   
95      1.311211e+09        1.0         1.0              2.0   
97     -1.825978e+09        0.0         

"\nML1_season1=np.round(mapperdf.fit_transform(ds2_spring.copy()), 2)\nfilename1 = 'ML1_season1.sav'\npickle.dump(ML1_season1, open(filename1, 'wb'))\nprint(ML1_season1,' saved')\n    \n\nML1_season3=np.round(mapperdf.fit_transform(ds2_autumn.copy()), 2)\nfilename3 = 'ML1_season3.sav'\npickle.dump(ML1_season3, open(filename3, 'wb'))\nprint(ML1_season3,' saved')\n        \n\nML1_season4=np.round(mapperdf.fit_transform(ds2_winter.copy()), 2)\nfilename4 = 'ML1_season4.sav'\npickle.dump(ML1_season4, open(filename4, 'wb'))\nprint(ML1_season4,' saved')\n"

In [214]:
# load the data from disk  ex: summer
dfdata = pickle.load(open(filename2, 'rb'))
#result = loaded_model.score(X_test, Y_test)
print('model loaded')

model loaded


In [220]:
dfdata.head(1)

,user_id,is_mobile,is_package,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,prop_is_branded,prop_starrating_0,prop_starrating_1,prop_starrating_2,prop_starrating_3,prop_starrating_4,prop_starrating_5,distance_band_C,distance_band_F,distance_band_M,distance_band_VC,distance_band_VF,hist_price_band_H,hist_price_band_L,hist_price_band_M,hist_price_band_VH,hist_price_band_VL,popularity_band_H,popularity_band_L,popularity_band_M,popularity_band_VH,cnt,month_ss,day_ss,hour_ss,month_ci,day_ci,month_co,day_co,season_cat,LOS,is_satsun_ss,is_worktime_ss,is_satsun,is_monfri,BW_ss,region_dest,user_cat_bleisure_row,user_cat_bleisure_usa,user_cat_business_row,user_cat_business_usa,user_cat_familynokids_row,user_cat_familynokids_usa,user_cat_familywkids_row,user_cat_familywkids_usa,user_cat_single_row,user_cat_single_usa,user_cat_wegetaway_row,user_cat_wegetaway_usa,channel_cat_direct,channel_cat_google_srch,channel_cat_mail,channel_cat_other,channel_cat_paid_srch,channel_cat_tripadvisor?,channel_cat_trivago?,income_hh,year_rank_ss,season_dest_rank_ss,timedelta_ss,package_income_t,mobile_BW_t,is_booking,is_churns,is_prebookings
65,-1.825978e+09,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,3.0,6.0,6.0,5.0,7.0,4.0,0.0,6.0,0.0,0.0,1.0,1.0,2.1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18638.0,1.0,1.0,365.0,18638.0,0.0,0.0,1.0,0.0


In [138]:
sample=ds2.iloc[0,0:37]
sample= pd.DataFrame(sample)
print(sample[:5])
len(sample)
type(sample)


                            0
user_id            -803007886
is_mobile                   0
is_package                  0
srch_adults_cnt             2
srch_children_cnt           0


pandas.core.frame.DataFrame

In [189]:
mapper.transformed_names_

['user_id',
 'is_mobile',
 'is_package',
 'srch_adults_cnt',
 'srch_children_cnt',
 'srch_rm_cnt',
 'prop_is_branded',
 'prop_starrating_0',
 'prop_starrating_1',
 'prop_starrating_2',
 'prop_starrating_3',
 'prop_starrating_4',
 'prop_starrating_5',
 'distance_band_C',
 'distance_band_F',
 'distance_band_M',
 'distance_band_VC',
 'distance_band_VF',
 'hist_price_band_H',
 'hist_price_band_L',
 'hist_price_band_M',
 'hist_price_band_VH',
 'hist_price_band_VL',
 'popularity_band_H',
 'popularity_band_L',
 'popularity_band_M',
 'popularity_band_VH',
 'cnt',
 'month_ss',
 'day_ss',
 'hour_ss',
 'month_ci',
 'day_ci',
 'month_co',
 'day_co',
 'season_cat',
 'LOS',
 'is_satsun_ss',
 'is_worktime_ss',
 'is_satsun',
 'is_monfri',
 'BW_ss',
 'region_dest',
 'user_cat_bleisure_row',
 'user_cat_bleisure_usa',
 'user_cat_business_row',
 'user_cat_business_usa',
 'user_cat_familynokids_row',
 'user_cat_familynokids_usa',
 'user_cat_familywkids_row',
 'user_cat_familywkids_usa',
 'user_cat_single_r

In [232]:
# Split data set (X_train, y_train and X_test, y_test)  # 

# helper function to split on user_id
def random_users_df(dfsplitcol,seed,split,split_type='test'):   
    users = np.unique(dfsplitcol.values )
    num_users = dfsplitcol.nunique()
    num_users_test=round(num_users*split)
    np.random.RandomState(seed)
    random_users_test = np.random.choice(users, replace=False, size=num_users_test)
    tdf=pd.DataFrame(random_users_test)
    tdf.columns=['user_id']
    
    print('unique users:',num_users,' ',split_type,' ',split,':', num_users_test,' train',1-split,':', num_users-num_users_test)
    return tdf

### REVIEW: this should be based on the dfDATA in binary format , aka OUTPUT of the MAPPER
# replace DATA

tds1=random_users_df(dfdata.user_id,2, 20/100,'test')

ds_test=pd.merge(dfdata,tds1,how='inner',on='user_id')
ds_train=dfdata[~dfdata.user_id.isin(tds1.user_id)]

y_test =ds_test.loc[:,'is_booking':'is_prebookings']  # col vector instead of pd.series
y_train=ds_train.loc[:,'is_booking':'is_prebookings']

remove_col = ['is_booking','is_churns','is_prebookings' ]  

X_test=ds_test.drop(remove_col, axis=1)
X_train=ds_train.drop(remove_col, axis=1)

print()
print('train: ',X_train.shape, y_train.shape)
print()
print('test:', X_test.shape, y_test.shape)

unique users: 33791   test   0.2 : 6758  train 0.8 : 27033

train:  (243099, 68) (243099, 3)

test: (63339, 68) (63339, 3)


In [226]:
type(X_train)

pandas.core.frame.DataFrame

In [204]:
sample=ds2.iloc[0,0:36]
sample

user_id                      -803007886
is_mobile                             0
is_package                            0
srch_adults_cnt                       2
srch_children_cnt                     0
srch_rm_cnt                           1
srch_destination_id             5527237
hotel_id                      153390727
prop_is_branded                       1
prop_starrating                       3
distance_band                         C
hist_price_band                       H
popularity_band                       H
cnt                                   1
month_ss                              1
day_ss                                3
hour_ss                               0
month_ci                              1
day_ci                                3
month_co                              1
day_co                                4
season_cat                    is_winter
LOS                                   1
is_satsun_ss                          0
is_worktime_ss                        0


### 2 Modelling

In [ ]:
# training a DescisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train)
dtree_predictions = dtree_model.predict(X_test)
 
# creating a confusion matrix
cm = confusion_matrix(y_test, dtree_predictions)

In [ ]:
# importing necessary libraries
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
 
# loading the iris dataset
iris = datasets.load_iris()
 
# X -> features, y -> label
X = iris.data
y = iris.target
 
# dividing X, y into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
 
# training a linear SVM classifier
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test)
 
# model accuracy for X_test  
accuracy = svm_model_linear.score(X_test, y_test)
 
# creating a confusion matrix
cm = confusion_matrix(y_test, svm_predictions)

In [236]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class='multinomial')
model.fit(X_train, y_train)

#score = model.score(X_test, y_test)
#print(score)

ValueError: bad input shape (243099, 3)

In [ ]:
In a for-loop from i=1 to K, do the following: 
Select the i'th fold as my CV set, and the remaining samples as my training set . 
Feature selection 1: Man Whitney test on the distribution (remove features not correlated with the target variable)
Feature selection 2: Collinearity test: we want to classify observations and interpret results. 
Feature selection 3: Ridge regression to add the L2 norm penalty (α∑ni=1w2i) to the loss function. Ridge instead of Lasso to select predictive features which will get a non zero coefficient.
Convert both CV set (the current fold, called CVi) and current training set Ti to one with the appropriate features. 
Train model on the training set Ti
Get the score from the current fold, CVi. Append this score to a list holding all the scores. 
Average the score to get the K-folds score. 

### 3 Visualization of results 

In [ ]:
# Option 1: use standard ROC curve

Here is a python code :

import matplotlib.pyplot as plt
import numpy as np

score = np.array([0.9, 0.8, 0.7, 0.6, 0.55, 0.54, 0.53, 0.52, 0.51, 0.505, 0.4, 0.39, 0.38, 0.37, 0.36, 0.35, 0.34, 0.33, 0.30, 0.1])
y = np.array([1,1,0, 1, 1, 1, 0, 0, 1, 0, 1,0, 1, 0, 0, 0, 1 , 0, 1, 0])

roc_x = []
roc_y = []
min_score = min(score)
max_score = max(score)
thr = np.linspace(min_score, max_score, 30)
FP=0
TP=0
N = sum(y)
P = len(y) - N

for (i, T) in enumerate(thr):
    for i in range(0, len(score)):
        if (score[i] > T):
            if (y[i]==1):
                TP = TP + 1
            if (y[i]==0):
                FP = FP + 1
    roc_x.append(FP/float(N))
    roc_y.append(TP/float(P))
    FP=0
    TP=0

plt.scatter(roc_x, roc_y)
plt.show()

In [28]:
# predict a probability for the new class 
Xnew = [[...], [...]]
ynew = model.predict_proba(Xnew)

['C']

In [ ]:
#### Feature selection and training

### Appendix

In [162]:
## Transform all features into numeric for ML  USE THIS FOR STANDARD SCALER

# 1 one hot encoder: 
# Encode categorical integer features using a one-hot aka one-of-K scheme.
# the input to this transformer should be a matrix of integers
# denoting the values taken on by categorical (discrete) features from 1 to n.

# label_binarizer:
# a one-hot encoding of y labels should use a LabelBinarizer instead
# needed to convert multi-class labels to binary labels (belong or does not belong to the class).
# LabelBinarizer makes this process easy with the transform method.

# label_binarize for fixed set of class labels known ahead of time.
          
features = [('user_id',                   None),
            ('is_mobile',                 None),
            ('is_package',                None),
            (['srch_adults_cnt'],         None),          # [StandardScaler()])
            (['srch_children_cnt'],       None),          # [StandardScaler()])
            (['srch_rm_cnt'],             None),          # [StandardScaler()])
            ('srch_destination_id',       None),
            ('hotel_id',                  None),
            ('prop_is_branded',           None),
            ('prop_starrating',[LabelBinarizer()]),
            ('distance_band',  [LabelBinarizer()]),
            ('hist_price_band',[LabelBinarizer()]),
            ('popularity_band',[LabelBinarizer()]),
            (['cnt'],                     None),          # [StandardScaler()])
            (['month_ss'],                None),          # [StandardScaler()])
            (['day_ss'],                  None),          # [StandardScaler()])
            (['hour_ss'],        [StandardScaler()]), 
            (['month_ci'],                None),          # [StandardScaler()])
            (['day_ci'],                  None),          # [StandardScaler()])
            (['month_co'],                None),          # [StandardScaler()])
            (['day_co'],                  None),          # [StandardScaler()])
            ('season_cat',     [LabelBinarizer()]),
            (['LOS'],                     None),          # [StandardScaler()])
            ('is_satsun_ss',              None),
            ('is_worktime_ss',            None),
            ('is_satsun',                 None),
            ('is_monfri',                 None),
            (['BW_ss'],     [StandardScaler()]),
            ('region_dest', [LabelBinarizer()]),
            ('user_cat',    [LabelBinarizer()]),
            ('channel_cat', [LabelBinarizer()]), 
            (['income_hh'], [StandardScaler()]),
            (['year_rank_ss'], [StandardScaler()]),
            (['season_dest_rank_ss'],[StandardScaler()]),
            (['timedelta_ss'],        [StandardScaler()]),
            ('package_income_t',          None),
            ('mobile_BW_t',               None), 
            ('is_booking',                None),
            ('is_churns',                 None),
            ('is_prebookings',            None), 
            ]


mapper1= DataFrameMapper(features, default=False,df_out=True) # drops unselected col
mapperdf1=DataFrameMapper(features, default=False,df_out=True) # mapper to split train and test

###### Definitions: 
1 sklearn's transform's fit() 
calculates the parameters (e.g. μ and σ in case of StandardScaler)
saves them as an internal objects state.

2 transform() 
method to apply the transformation to a particular set of examples.

3 fit_transform() joins these two steps 
is used for the initial fitting of parameters on the training set x, but it also returns a transformed x′. 

4 inverse_transform(Y, threshold=None): transform binaries into multi class label

5 get_params(deep=True)[source]: get parameters of the model

###### note on scaling
1 "monotonic transformation" invariant learning methods: decision trees + derivations
==> Not affected by scaling

2 translation invariant learning methods (kNN, SVM with RBF kernel)
==> With the translation invariant algorithms, centering is useless. 

3 others.

Now, for the other methods, it really depends on the data. 
Usually worth trying with scaling (especially if variables have different orders of magnitude).
In a general case, I would recommend trying various preprocessings of the data : 
without scaling
scaling dividing by the standard deviation
scaling dividing by the sum of absolute values of your data (which would make it lie on a simplex). 
One of them will perform better than the others, but I cannot say which one until I have tried.

In [ ]:
## Transform all features into numeric for ML

# 1 one hot encoder: 
# Encode categorical integer features using a one-hot aka one-of-K scheme.
# the input to this transformer should be a matrix of integers
# denoting the values taken on by categorical (discrete) features from 1 to n.

# label_binarizer:
# a one-hot encoding of y labels should use a LabelBinarizer instead
# needed to convert multi-class labels to binary labels (belong or does not belong to the class).
# LabelBinarizer makes this process easy with the transform method.

# label_binarize for fixed set of class labels known ahead of time.
          
features = [('user_id',                   None),
            ('is_mobile',                 None),
            ('is_package',                None),
            (['srch_adults_cnt'],    [StandardScaler()]),         
            (['srch_children_cnt'],  [StandardScaler()]),    
            (['srch_rm_cnt'],        [StandardScaler()]),      
            ('srch_destination_id',       None),
            ('hotel_id',                  None),
            ('prop_is_branded',           None),
            ('prop_starrating',[LabelBinarizer()]),
            ('distance_band',  [LabelBinarizer()]),
            ('hist_price_band',[LabelBinarizer()]),
            ('popularity_band',[LabelBinarizer()]),
            (['cnt'],          [StandardScaler()]),           
            (['month_ss'],     [StandardScaler()]),            
            (['day_ss'],       [StandardScaler()]),     
            (['hour_ss'],        [StandardScaler()]), 
            (['month_ci'],                None),         
            (['day_ci'],        [StandardScaler()]), 
            (['month_co'],      [StandardScaler()]), 
            (['day_co'],        [StandardScaler()]),
            ('season_cat',      [LabelBinarizer()]),
            (['LOS'],           [StandardScaler()]),
            ('is_satsun_ss',              None),
            ('is_worktime_ss',            None),
            ('is_satsun',                 None),
            ('is_monfri',                 None),
            (['BW_ss'],     [StandardScaler()]),
            ('region_dest', [LabelBinarizer()]),
            ('user_cat',    [LabelBinarizer()]),
            ('channel_cat', [LabelBinarizer()]), 
            (['income_hh'], [StandardScaler()]),
            (['year_rank_ss'],        [StandardScaler()]),
            (['season_dest_rank_ss'], [StandardScaler()]),
            (['timedelta_ss'],        [StandardScaler()]),
            ('package_income_t',          None),
            ('mobile_BW_t',               None), 
            ('is_booking',                None),
            ('is_churns',                 None),
            ('is_prebookings',            None), 
            ]


mapper1= DataFrameMapper(features, default=False,df_out=True) # drops unselected col

#ds_ML1=mapper1.fit_transform(X_train)


In [ ]:
slice(None) creates a slice instance as slice(None, None, None) 
with default for start, step and end (see [1]). 
Wrt your code snippet, the tuple returns the individual computed means
on the second axis as per [2]. Let's confirm this:

###### note on k-fold
Need to use sklearn stratified Kfold to take into account class imbalance

In [60]:
## k-fold : review iterator definition with Timo if could be applicable

def split_fold2(fold_data, n_folds, new_fold_col='fold'): # enter fold data, nb fold and fold col
    i_fold=0  # var initialized at 0
    indices=list(fold_data.index.names)  # creates a list with data index.names names of index(es)
    slicers=[slice(None)]*len(fold_data.index.names)  # [slice(np.newaxis)]*len ==>ex: 2*[slice on new axis] 
    fold_index=fold_data.index.names.index(new_fold_col) # multi index ex ID1 1, ID1 2, IDn 3... if ID in new_fold_col
    indices.remove(new_fold_col) # remove the variable new_fold_col (re-initialize indices)

    while (i_fold<n_folds):    # as long as i_fold < n_folds (max)
        slicers[fold_index]=[i for i in range(n_folds) if i !=i_fold]   # n-1 fold except i_fold
        slicers_tuple=tuple(slicers)         # create a tuple from slicers  ex: ( 0,1) ?                      
        train_data=fold_data.loc[slicers_tuple,:].groupby(level=indices).sum()  
        # select train data from fold_data.loc, grouping by level =indices of which we do the sum...
        
        val_data=fold_data.xs(i_fold,level=new_fold_col)  # get validation data from cross section of data)
        yield train_data,val_data
        i_fold+=1
        
##note: 
## source: https://stackoverflow.com/questions/40369987/sklearn-cross-validation-for-grouped-data/40370168
#DataFrame.xs(key, axis=0, level=None, drop_level=True)[source]
# Returns a cross-section (row(s) or column(s)) from the Series/DataFrame.
# Defaults to cross-section on the rows (axis=0).

In [27]:
### test returns name(s) of an index Multiindex
dftest1000 = pd.DataFrame({'A': [1, 2, 3], 'B': [1, 1, 1], 'C':['F','G','H']})
dftest1000
dftest1000.index=dftest1000['C']
dftest1000

indices=list(dftest1000.index.names)
indices

,A,B,C
C,,,
F,1,1,F
G,2,1,G
H,3,1,H


In [29]:
a = [::]*2
a

SyntaxError: invalid syntax (<ipython-input-29-76985201b887>, line 1)